In [ ]:
from matplotlib import pyplot
%matplotlib inline
pyplot.plot([1,2,3])

In [ ]:
import os
from collections import defaultdict
from matplotlib import pyplot
import torch
import numpy as np
import scipy.stats
from torch.distributions import constraints
import pandas as pd

import pyro
import pyro.distributions as dist
from pyro import poutine
from pyro.infer.autoguide import AutoDelta,AutoNormal
from pyro.optim import Adam
from pyro.infer import SVI, TraceEnum_ELBO, Trace_ELBO, config_enumerate, infer_discrete

import random


In [ ]:
pi = [1/2,1/3,1/6]
mu = [1.0,5.0,10.0]
sigma2 = [1.0,1.5,2.0]

In [ ]:
n_samples = 10000
data = []

#first we generate n_samples samples from the distribution pi i.e. a sequece of the form {1,3,1,2,1,3,3,2,.....}
cluster_assignments = np.random.choice(3,n_samples,p=pi)


# for each of the indices in cluster_assignments we sample from the Normal(mu[idx],sigma2[idx]) distribution
for assignment in cluster_assignments:
    mean = mu[assignment]
    var = sigma2[assignment]
    sample = np.random.normal(mean,var)
    data.append(sample)

In [ ]:
# for convenience we structure the data set as a pandas DataFrame
df = pd.DataFrame(data=data,columns=['data'])

In [ ]:
# We need the data as a tensor
data = torch.tensor(data)

In [ ]:
data.shape

In [ ]:
#this code displays the histogram of the data set with a smooth curve overlaid
import seaborn as sns

pyplot.rcParams["figure.figsize"] = (14,14)

ax = df['data'].plot.hist(bins=100, density=True, edgecolor='w', linewidth=2.5)

# Save default x-axis limits for final formatting because the pandas kde
# plot uses much wider limits which usually decreases readability
xlim = ax.get_xlim()

# Plot pandas KDE
df['data'].plot.density(color='r', linewidth = 10,alpha=1, ax=ax) # same as df['var'].plot.kde()

# Reset x-axis limits and edit legend and add title
ax.set_xlim(xlim)
ax.legend(labels=['KDE'], frameon=False)
ax.set_title('Pandas histogram overlaid with KDE', fontsize=14, pad=15)

pyplot.show()

In [ ]:
# we first have to define the pyro model. We first specify the prior. There are three groups of parameters: the weights i.e.
# the distribution over {0,1,2}, the means for each of the tree Gaussians and their variances. These are all independent so
# the prior is a product of the distributions over each parameter
K=3

@config_enumerate
def model(data):
    samples = []
    # the distribution pi is sampled from a Dirichlet distribution. The Dirichlet is a distribution over the
    # probability simplex i.e. all vectors $x_1,x_2,...,x_n$ with x_i > 0 and summing up to 1
    # each of the parameters are stored in the pyro.param_store as items in a dict, as we can see they all have to be named.
    # In pyro distributions are basically characterized by their samples
    weights=pyro.sample('weights',dist.Dirichlet(0.5*torch.ones(K)))
    
    #the pyro.plate specifies an array of independent distributions. Here the distribution for each of the means is a
    #Gaussian with mean 0 and variance 10, the variances have a LogNormal distribution since the have to be > 0.
    # these distributions are stored in an array in the param_store named 'components'
    with pyro.plate('components',K):
        #the prior distriburion of the means
        locs = pyro.sample('locs',dist.Normal(0.,10.))
        # the prior distribution of the variances
        scales = pyro.sample('scales',dist.LogNormal(0.,2.))
        
    #here is the likelihood, for each data point an assignment to a Gaussian and then it is a sample from that Gaussian
    # first a sample from the distribution pi, which in turn is a sample from the Dirichlet distribution above.
    # then a sample from the corresponding Gaussian with mean and variance sampled from the appropriate priors
    with pyro.plate('data',len(data)):
        assignment = pyro.sample('assignment',dist.Categorical(weights))
        pyro.sample('obs',dist.Normal(locs[assignment],scales[assignment]),obs=data)
        

In [ ]:
pi = [1/2,1/3,1/6]
mu = [1.0,5.0,10.0]
sigma2 = [1.0,1.5,2.0]

In [ ]:
# we import the appropriate pyro packages

from pyro.infer.mcmc.api import MCMC
from pyro.infer.mcmc import HMC,NUTS,MCMC
pyro.set_rng_seed(2)
kernel = NUTS(model)
mcmc = MCMC(kernel,num_samples=1000,warmup_steps=50)
mcmc.run(data)
posterior_samples = mcmc.get_samples()

In [ ]:
posterior_samples.keys()

In [ ]:
posterior_samples['locs'].numpy()[:,0]

In [ ]:
pyplot.hist(posterior_samples['locs'].numpy()[:,0],bins=100,density=True);
pyplot.hist(posterior_samples['locs'].numpy()[:,1],bins=100,density=True);
pyplot.hist(posterior_samples['locs'].numpy()[:,2],bins=100,density=True);

In [ ]:
pyplot.hist(posterior_samples['scales'].numpy()[:,0],bins=100,density=True);
pyplot.hist(posterior_samples['scales'].numpy()[:,1],bins=100,density=True);
pyplot.hist(posterior_samples['scales'].numpy()[:,2],bins=100,density=True);

In [ ]:
pyplot.hist(posterior_samples['weights'].numpy()[:,0],bins=100,density=True);
pyplot.hist(posterior_samples['weights'].numpy()[:,1],bins=100,density=True);
pyplot.hist(posterior_samples['weights'].numpy()[:,2],bins=100,density=True);

In [ ]:
posterior_samples['locs'][500:].mean(0)

In [ ]:
posterior_samples['scales'].numpy()[500:].mean(0)

In [ ]:
posterior_samples['weights'].numpy()[500:].mean(0)